In [1]:
from typing import Dict, Text

import os
import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2024-02-02 18:54:10.399633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/alexlee/miniconda3/envs/hkjc-tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

ratings

<_MapDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [69]:
df = pd.read_csv(os.path.join('data','all_races_cleaned.csv'), index_col=0)

ratings = pd.DataFrame()
ratings['movie_title'] = df['horse_colour'].astype(str)
ratings['user_id'] = df['race_class'].astype(str)
ratings['user_rating'] = df['place'].astype('float32')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))
# ratings = tf.cast(ratings, tf.string)

movies = pd.DataFrame()
movies['movie_title'] = df['horse_colour'].astype(str)
movies = tf.data.Dataset.from_tensor_slices(dict(movies))
# movies = tf.cast(movies, tf.string)

ratings

<_TensorSliceDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [70]:
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(0.8*len(shuffled)))
test = shuffled.skip(int(0.8*len(shuffled))).take(int(0.2*len(shuffled)))

In [71]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [72]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [73]:
RankingModel()((["1000000"], ["griffin"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00255738]], dtype=float32)>

In [74]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [75]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [76]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [77]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [78]:
model.fit(cached_train, epochs=3)

Epoch 1/3


3/3 [==============================] - 1s 13ms/step - root_mean_squared_error: 7.9669 - loss: 189.1026 - regularization_loss: 0.0000e+00 - total_loss: 189.1026
Epoch 2/3
3/3 [==============================] - 0s 9ms/step - root_mean_squared_error: 6.9917 - loss: 44.1954 - regularization_loss: 0.0000e+00 - total_loss: 44.1954
Epoch 3/3
3/3 [==============================] - 0s 11ms/step - root_mean_squared_error: 4.5736 - loss: 25.5883 - regularization_loss: 0.0000e+00 - total_loss: 25.5883


In [79]:
model.evaluate(cached_test, return_dict=True)

2/2 [==============================] - 0s 3ms/step - root_mean_squared_error: 4.2722 - loss: 17.2823 - regularization_loss: 0.0000e+00 - total_loss: 17.2823


{'root_mean_squared_error': 4.2721686363220215,
 'loss': 16.74215316772461,
 'regularization_loss': 0,
 'total_loss': 16.74215316772461}

In [86]:
test_ratings = {}
test_movie_titles = ['bay','brown','chestnut','grey']
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["1"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
bay: [[3.6322863]]
brown: [[3.051286]]
chestnut: [[2.7532363]]
grey: [[2.6427293]]
